In [1]:
import numpy as np
import pandas as pd
dataset = pd.read_csv('dataset.csv')


In [2]:
print(dataset.head())

   step      type    amount     nameOrig  oldbalanceOrg  newbalanceOrig   
0     1   PAYMENT   9839.64  C1231006815       170136.0       160296.36  \
1     1   PAYMENT   1864.28  C1666544295        21249.0        19384.72   
2     1  TRANSFER    181.00  C1305486145          181.0            0.00   
3     1  CASH_OUT    181.00   C840083671          181.0            0.00   
4     1   PAYMENT  11668.14  C2048537720        41554.0        29885.86   

      nameDest  oldbalanceDest  newbalanceDest  isFraud  isFlaggedFraud  
0  M1979787155             0.0             0.0        0               0  
1  M2044282225             0.0             0.0        0               0  
2   C553264065             0.0             0.0        1               0  
3    C38997010         21182.0             0.0        1               0  
4  M1230701703             0.0             0.0        0               0  


In [3]:
print(dataset['type'].value_counts())


type
CASH_OUT    2237500
PAYMENT     2151495
CASH_IN     1399284
TRANSFER     532909
DEBIT         41432
Name: count, dtype: int64


In [4]:
corrolation = dataset[dataset.columns.difference(['type','nameOrig','nameDest'])].corr()
print(corrolation['isFraud'])


amount            0.076688
isFlaggedFraud    0.044109
isFraud           1.000000
newbalanceDest    0.000535
newbalanceOrig   -0.008148
oldbalanceDest   -0.005885
oldbalanceOrg     0.010154
step              0.031578
Name: isFraud, dtype: float64


In [5]:
dataset['type'] = dataset['type'].map({"CASH_OUT":1, "PAYMENT":2, "CASH_IN":3,"TRANSFER":4,"DEBIT":5})

In [6]:
dataset["isFlaggedFraud"].value_counts()


isFlaggedFraud
0    6362604
1         16
Name: count, dtype: int64

In [7]:
frauders = dataset.loc[dataset['isFraud']==1]
frauders['type'].value_counts()

type
1    4116
4    4097
Name: count, dtype: int64

In [8]:
X = dataset.loc[(dataset.type == 1) | (dataset.type == 4)]
X.loc[X.type == 4, 'type'] = 0
Y = X['isFraud']
X.drop(columns=['isFraud', 'isFlaggedFraud','nameOrig','nameDest'], inplace=True)

/tmp/ipykernel_4146/3570810444.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.drop(columns=['isFraud', 'isFlaggedFraud','nameOrig','nameDest'], inplace=True)


In [9]:
X.loc[(X.oldbalanceDest == 0) & (X.newbalanceDest == 0) & (X.amount != 0),
      ['oldbalanceDest', 'newbalanceDest']] = - 1


In [10]:
X.loc[(X.oldbalanceOrg == 0) & (X.newbalanceOrig == 0) & (X.amount != 0), 
      ['oldbalanceOrg', 'newbalanceOrig']] = np.nan


In [11]:
X['errorbalanceOrig'] = X.newbalanceOrig + X.amount - X.oldbalanceOrg
X['errorbalanceDest'] = X.oldbalanceDest + X.amount - X.newbalanceDest


/tmp/ipykernel_4146/4205422170.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['errorbalanceOrig'] = X.newbalanceOrig + X.amount - X.oldbalanceOrg
/tmp/ipykernel_4146/4205422170.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['errorbalanceDest'] = X.oldbalanceDest + X.amount - X.newbalanceDest


In [12]:
from sklearn.model_selection import train_test_split
trainX, testX, trainY, testY = train_test_split(X, Y, test_size = 0.2, random_state = 4)

In [23]:
from xgboost import XGBClassifier


weights = (Y == 0).sum()/(Y==1).sum()
clf = XGBClassifier(max_depth = 3, scale_pos_weight = weights, n_jobs = 4)
clf.fit(trainX, trainY)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=4, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [31]:
from sklearn.metrics import average_precision_score,f1_score
probabilities = clf.predict_proba(testX)
pred = clf.predict(testX)
print(average_precision_score(testY, probabilities[:,1]))
print (f1_score(pred,testY))

0.9965021460859849
0.9926405652045922
